<a href="https://www.kaggle.com/code/gregoryoliveira/brazil-weather-change-part-i-data-collection?scriptVersionId=105744176" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 1. Introduction

This notebook is part of [Applied Data Science with Python Specialization](https://www.coursera.org/specializations/data-science-python), especially the 2nd course [Applied Plotting, Charting & Data Representation in Python](https://www.coursera.org/learn/python-plotting?specialization=data-science-python).

"This assignment requires that you to find at least two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of weather phenomena (see below) for the region of Juiz de Fora, Minas Gerais, Brazil, or Brazil more broadly."

I choose to create a notebook that use [INMET](https://portal.inmet.gov.br/dadoshistoricos) (National Institute of Meteorology - Instituto Nacional de Meteorologia) to compare some measures between brazilian cities. This notebook was first built with Kaggle dataset [Brazil Weather, Automatic Stations (2000-2021)](https://www.kaggle.com/datasets/saraivaufc/automatic-weather-stations-brazil). However, considering that I want 2021 data updated, I code [the first part](https://www.kaggle.com/code/gregoryoliveira/brazil-weather-change-part-i-data-collection) to collect the data and summarize. The data is also avaiable as [Brazil Weather Information by INMET](https://www.kaggle.com/datasets/gregoryoliveira/brazil-weather-information-by-inmet).

In [1]:
# Libraries
import os
import requests
import fnmatch
import pandas as pd
import numpy as np
from datetime import date
from zipfile import ZipFile
from IPython.display import HTML

In [2]:
# Creating the table with information
df_vars = pd.DataFrame(
    data={
        'Portuguese': ['ESTACAO', 'DATA', 'HORA', 'PRECIPITACAO TOTAL HORARIO',
                       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA',
                       'PRESSAO ATMOSFERICA MAX.NA HORA ANT.',
                       'PRESSAO ATMOSFERICA MIN. NA HORA ANT.', 'RADIACAO GLOBAL',
                       'TEMPERATURA DO AR - BULBO SECO, HORARIA','TEMPERATURA DO PONTO DE ORVALHO',
                       'TEMPERATURA MAXIMA NA HORA ANT.', 'TEMPERATURA MINIMA NA HORA ANT.',
                       'TEMPERATURA ORVALHO MAX. NA HORA ANT.', 'TEMPERATURA ORVALHO MIN. NA HORA ANT.',
                       'UMIDADE REL. MAX. NA HORA ANT.', 'UMIDADE REL. MIN. NA HORA ANT.',
                       'UMIDADE RELATIVA DO AR, HORARIA', 'VENTO, DIRECAO HORARIA',
                       'VENTO, RAJADA MAXIMA', 'VENTO, VELOCIDADE HORARIA'],
        'English': ['STATION', 'DATE', 'TIME', 'TOTAL HOURLY RAIN',
                    'ATMOSPHERIC PRESSURE AT STATION LEVEL, TIME',
                    'MAX. ATMOSPHERIC PRESSURE IN THE PREVIOUS TIME.',
                    'ATMOSPHERIC PRESSURE MIN. IN THE EARLY TIME.', 'GLOBAL RADIATION',
                    'AIR TEMPERATURE - DRY BULB, TIME', 'DEW POINT TEMPERATURE',
                    'MAXIMUM TEMPERATURE IN THE PREVIOUS TIME.', 'MINIMUM TEMPERATURE IN THE PREVIOUS TIME.',
                    'MAX DEW TEMPERATURE. IN THE EARLY TIME.', 'DEW TEMPERATURE MIN. IN THE EARLY TIME.',
                    'REL HUMIDITY MAX. IN THE EARLY TIME.', 'REL HUMIDITY MIN IN THE EARLY TIME.',
                    'RELATIVE AIR HUMIDITY, HOURS', 'WIND, TIME DIRECTION', 'WIND, MAXIMUM GUNS',
                    'WIND, HOUR SPEED'],
        'Measure' : [' ', 'YYYY-MM-DD',
                     'UTC', 'mm', 'mB',
                     'mB', 'mB', 'W/m²',
                     'ºC', 'ºC', 'ºC',
                     'ºC', 'ºC', 'ºC',
                     '%', '%', '%',
                     'gr', 'm/s', 'm/s'],
        'Select' : ['', '', '', 'Yes', '', '', '', 'Yes', 'Yes', '', 'Yes', 'Yes', '', '', 'Yes', 'Yes', 'Yes', '', 'Yes', 'Yes'],
        'Aggreation' : ['', '', '', 'Maximum', '', '', '',
                        'Maximum', 'Average', '', 'Maximum', 'Minimum',
                        '', '', 'Maximum', 'Minimum', 'Average', '',
                        'Maximum', 'Average'],
        'Column Name' : ['', '', '', 'rain_max', '', '', '',
                         'rad_max', 'temp_avg', '', 'temp_max',
                         'temp_min', '', '', 'hum_max', 'hum_min',
                         'hum_avg', '', 'wind_max', 'wind_avg', ]
    }
)

# Showing
HTML(df_vars.to_html(escape=False,index=False, justify='center'))

Portuguese,English,Measure,Select,Aggreation,Column Name
ESTACAO,STATION,,,,
DATA,DATE,YYYY-MM-DD,,,
HORA,TIME,UTC,,,
PRECIPITACAO TOTAL HORARIO,TOTAL HOURLY RAIN,mm,Yes,Maximum,rain_max
"PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA","ATMOSPHERIC PRESSURE AT STATION LEVEL, TIME",mB,,,
PRESSAO ATMOSFERICA MAX.NA HORA ANT.,MAX. ATMOSPHERIC PRESSURE IN THE PREVIOUS TIME.,mB,,,
PRESSAO ATMOSFERICA MIN. NA HORA ANT.,ATMOSPHERIC PRESSURE MIN. IN THE EARLY TIME.,mB,,,
RADIACAO GLOBAL,GLOBAL RADIATION,W/m²,Yes,Maximum,rad_max
"TEMPERATURA DO AR - BULBO SECO, HORARIA","AIR TEMPERATURE - DRY BULB, TIME",ºC,Yes,Average,temp_avg
TEMPERATURA DO PONTO DE ORVALHO,DEW POINT TEMPERATURE,ºC,,,


Before we get our hands dirty, I would like to thanks Samuel Oranyeli. The following lines was inspired on the text [Extract DataFrame from Compressed Data into Pandas](https://samukweku.github.io/data-wrangling-blog/python/pandas/compressed%20data/zip/archived%20data/2020/07/21/Extract-DataFrame-from-Compressed-Data-into-Pandas.html#Compressed-Data-Extraction-in-Pandas) that I found at his page.

Another special thanks for robinood. [This answer](https://stackoverflow.com/questions/65020294/downloading-a-zipped-file-using-python-3-7) give the function to download files.

In [3]:
# Functions
# Downloading file
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)
            
# Pandas read_csv for brazilian files. station parameters
def read_station(file_name):
    return pd.read_csv(
        zipfiles.open(file_name),
        encoding='latin-1',
        nrows=8, sep=';',
        header=None,
        decimal=','
    )

# Pandas read_csv for brazilian files. weather information
def read_weather(file_name):
    return pd.read_csv(
        zipfiles.open(file_name),
        encoding='latin-1',
        skiprows=8, sep=';',
        usecols=[x for x in range(0,19)],
        na_values=['-9999'],
        decimal=','
    )

# Cleaning and summarizing data
def df_clean_sum(x):
    return _data_weather[x].groupby(['ESTACAO','DATA (YYYY-MM-DD)']).agg(
            rain_max=pd.NamedAgg(
                column='PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
                aggfunc='max'
            ),
            rad_max=pd.NamedAgg(
                column='RADIACAO GLOBAL (KJ/m²)',
                aggfunc='max'
            ),
            temp_avg=pd.NamedAgg(
                column='TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
                aggfunc='mean'
            ),
            temp_max=pd.NamedAgg(
                column='TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
                aggfunc='max'
            ),
            temp_min=pd.NamedAgg(
                column='TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
                aggfunc='min'
            ),
            hum_max=pd.NamedAgg(
                column='UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
                aggfunc='max'
            ),
            hum_min=pd.NamedAgg(
                column='UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
                aggfunc='min'
            ),
            wind_max=pd.NamedAgg(
                column='VENTO, RAJADA MAXIMA (m/s)',
                aggfunc='max'
            ),
            wind_avg=pd.NamedAgg(
                column='VENTO, VELOCIDADE HORARIA (m/s)',
                aggfunc='mean'
            )
        ).dropna(how='all')

In [4]:
stations = pd.DataFrame()
weather = pd.DataFrame()
weather_sum = pd.DataFrame()

next_year = date.today().year + 1

headers = ['DATA (YYYY-MM-DD)',
 'Hora UTC',
 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
 'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
 'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
 'RADIACAO GLOBAL (KJ/m²)',
 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
 'TEMPERATURA DO PONTO DE ORVALHO (°C)',
 'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
 'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
 'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
 'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
 'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
 'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
 'UMIDADE RELATIVA DO AR, HORARIA (%)',
 'VENTO, DIREÇÃO HORARIA (gr) (° (gr))',
 'VENTO, RAJADA MAXIMA (m/s)',
 'VENTO, VELOCIDADE HORARIA (m/s)',
 'ESTACAO']

for year in range(2000,next_year):
    
    print('Starting to import the ' + str(year) + ' data...')
    
    url = 'https://portal.inmet.gov.br/uploads/dadoshistoricos/' + str(year) + '.zip' 

    save_path = './' + str(year) + '.zip'

    download_url(url,save_path)

    with ZipFile(save_path) as zipfiles:
        
        _file_list = zipfiles.namelist()
        
        _csv_files = fnmatch.filter(_file_list, "*.CSV")
        
        _data_station = [read_station(file_name).T for file_name in _csv_files]
        _data_weather = [read_weather(file_name) for file_name in _csv_files]
        
        # Filling the station
        for x in range(0,len(_data_station)):
            _data_weather[x]['ESTACAO'] = _data_station[x][3][1]
            _data_weather[x].columns = headers
            _data_weather[x].iloc[:, 0] = _data_weather[x].iloc[:, 0].astype('datetime64[ns]')
        
        # Summaring weaather information
        _data_weather_smm = [df_clean_sum(x) for x in range(0,len(_data_station))]

    os.remove(save_path)
    
    _stations = pd.concat(_data_station)
    _weather = pd.concat(_data_weather)
    _weather_sum = pd.concat(_data_weather_smm)
    
    # Exporting raw data by year
    save_file = 'weather_' + str(year) + '.csv'
    save_path = './' + 'weather_' + str(year) + '.zip'
    compression_opts = dict(method='zip',archive_name=save_file) 
    _weather.to_csv(save_path, index=False, compression=compression_opts)
    
    # Exporting summarize data by year
    save_file = 'weather_sum_' + str(year) + '.csv'
    save_path = './' + 'weather_sum_' + str(year) + '.zip'
    compression_opts = dict(method='zip',archive_name=save_file) 
    _weather_sum.reset_index().to_csv(save_path, index=False, compression=compression_opts)
    
    stations = pd.concat([stations, _stations])
    # weather = pd.concat([weather, _weather])
    weather_sum = pd.concat([weather_sum, _weather_sum])
    
    print('Finishing importing ' + str(year) + ' data.')
    print('...')
    
# Exporting raw data
#save_file = 'weather' + '.csv'
#save_path = './' + 'weather' + '.zip'
#compression_opts = dict(method='zip',archive_name=save_file) 
#weather.to_csv(save_path, index=False, compression=compression_opts)

# Exporting summarize data
save_file = 'weather_sum_all' + '.csv'
save_path = './' + 'weather_sum_all' + '.zip'
compression_opts = dict(method='zip',archive_name=save_file) 
weather_sum.reset_index().to_csv(save_path, index=False, compression=compression_opts)

Starting to import the 2000 data...
Finishing importing 2000 data.
...
Starting to import the 2001 data...
Finishing importing 2001 data.
...
Starting to import the 2002 data...
Finishing importing 2002 data.
...
Starting to import the 2003 data...
Finishing importing 2003 data.
...
Starting to import the 2004 data...
Finishing importing 2004 data.
...
Starting to import the 2005 data...
Finishing importing 2005 data.
...
Starting to import the 2006 data...
Finishing importing 2006 data.
...
Starting to import the 2007 data...
Finishing importing 2007 data.
...
Starting to import the 2008 data...
Finishing importing 2008 data.
...
Starting to import the 2009 data...
Finishing importing 2009 data.
...
Starting to import the 2010 data...
Finishing importing 2010 data.
...
Starting to import the 2011 data...
Finishing importing 2011 data.
...
Starting to import the 2012 data...
Finishing importing 2012 data.
...
Starting to import the 2013 data...
Finishing importing 2013 data.
...
Starti

In [5]:
# Finding last records
stations_last = weather_sum.reset_index()[['ESTACAO', 'DATA (YYYY-MM-DD)']]

stations_last = stations_last.groupby('ESTACAO').agg('max')

stations_last = stations_last.reset_index()

stations_last.columns = ['id_station', 'record_last']

# Cleaning stations DataFrame
headers = ['region', 'state', 'city_station', 'id_station', 'lat', 'lon', 'lvl', 'record_first']

stations.columns = headers

stations = stations[stations.index != 0]

stations.drop_duplicates(subset=['id_station'], inplace=True);

stations.reset_index(inplace=True);

stations = stations.merge(stations_last)

stations = stations.drop(columns='index')

stations['record_first'] = stations['record_first'].astype('datetime64[ns]')

stations.to_csv('./stations.csv', index=False)

## Before go

This is the first part. [The second part](https://www.kaggle.com/gregoryoliveira/brazil-weather-change-part-ii-data-analysis) is here at Kaggle! Hope you enjoy!